In [1]:
import os

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"]="2,1"  

In [2]:
# required libraries
!pip install matplotlib
!pip install pandas
!pip install torch torchvision torchaudio
!pip install protobuf --upgrade



In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

import torch
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer, BertForTokenClassification

c:\Users\nipua\AppData\Local\anaconda3\envs\ner_ast\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [5]:
data_path_train = "../Data/processed/merged/train.tsv" 
train_data = pd.read_csv(data_path_train, sep="\t").astype(str)

data_path_dev= "../Data/processed/merged/dev.tsv" 
dev_data = pd.read_csv(data_path_dev, sep="\t").astype(str)

data_path_test= "../Data/processed/merged/test.tsv" 
test_data = pd.read_csv(data_path_test, sep="\t").astype(str)

In [6]:
print(train_data.shape)

(134254, 4)


In [7]:
# Take top 50 rows from each DataFrame
train_data = train_data.head(500)
dev_data = dev_data.head(500)
test_data = test_data.head(500)

## Parser data

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                        #    s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    # def get_next(self):
    #     try:
    #         s = self.grouped["sentence: {}".format(self.n_sent)]
    #         self.n_sent += 1
    #         return s
    #     except:
    #         return None

In [9]:
# Get full document data struce
train_getter = SentenceGetter(train_data)
# Get sentence data
train_sentences = [[s[0] for s in sent] for sent in train_getter.sentences]
train_labels = [[s[1] for s in sent] for sent in train_getter.sentences]

C:\Users\nipua\AppData\Local\Temp\ipykernel_12060\2688801345.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("sentence #").apply(agg_func)


In [10]:
# Get full document data struce
dev_getter = SentenceGetter(dev_data)
# Get sentence data
dev_sentences = [[s[0] for s in sent] for sent in dev_getter.sentences]
dev_labels = [[s[1] for s in sent] for sent in dev_getter.sentences]

C:\Users\nipua\AppData\Local\Temp\ipykernel_12060\2688801345.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("sentence #").apply(agg_func)


In [11]:
# Get full document data struce
test_getter = SentenceGetter(test_data)
# Get sentence data
test_sentences = [[s[0] for s in sent] for sent in test_getter.sentences]
test_labels = [[s[1] for s in sent] for sent in test_getter.sentences]

C:\Users\nipua\AppData\Local\Temp\ipykernel_12060\2688801345.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("sentence #").apply(agg_func)


In [12]:
train_labels[20]

['O', 'O', 'O']

In [13]:
# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
tag2idx={'B-problem': 0,
 'B-test': 1,
 'B-treatment': 2,
 'I-problem': 3,
 'I-test': 4,
 'I-treatment': 5,
 'O': 6,
 'X': 7,
 '[CLS]': 8,
 '[SEP]': 9
 }
# Mapping index to name
tag2name = {tag2idx[key]: key for key in tag2idx}

In [14]:
tag2idx

{'B-problem': 0,
 'B-test': 1,
 'B-treatment': 2,
 'I-problem': 3,
 'I-test': 4,
 'I-treatment': 5,
 'O': 6,
 'X': 7,
 '[CLS]': 8,
 '[SEP]': 9}

In [15]:
tag2name

{0: 'B-problem',
 1: 'B-test',
 2: 'B-treatment',
 3: 'I-problem',
 4: 'I-test',
 5: 'I-treatment',
 6: 'O',
 7: 'X',
 8: '[CLS]',
 9: '[SEP]'}

## Make training data

Make raw data into trainable data for BERT, including:

- Set gpu environment
- Load tokenizer and tokenize
- Set 3 embedding, token embedding, mask word embedding, segmentation embedding
- Split data set into train and validate, then send them to dataloader

**Set up gpu environment**

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

### Load tokenizer

You can download the tokenizer file into local folder first :
- [vocab.txt](https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt)

In [17]:
# Len of the sentence must be not bigger than the training model
# See model's 'max_position_embeddings' = 512
max_len  = 128

In [18]:
# # model_name = "emilyalsentzer/Bio_ClinicalBERT"
# model_name = "google-bert/bert-base-uncased"
model_name = "dmis-lab/biobert-base-cased-v1.1"

In [19]:
tokenizer = BertTokenizer.from_pretrained(model_name)

**Tokenizer text**

- In hunggieface for bert, when come across OOV, will word piece the word
- We need to adjust the labels base on the tokenize result, “##abc” need to set label "X" 
- Need to set "[CLS]" at front and "[SEP]" at the end, as what the paper do, [BERT indexer should add [CLS] and [SEP] tokens](https://github.com/allenai/allennlp/issues/2141)


In [20]:
def process_data(sentences, labels):
    tokenized_texts = []
    word_piece_labels = []
    i_inc = 0
    for word_list,label in (zip(sentences,labels)):
        temp_lable = []
        temp_token = []
        
        # # Add [CLS] at the front 
        # temp_lable.append('[CLS]')
        # temp_token.append('[CLS]')
        
        for word, lab in zip(word_list,label):
            token_list = tokenizer.tokenize(word)
            for m,token in enumerate(token_list):
                temp_token.append(token)
                if m==0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')  
                    
        # # Add [SEP] at the end
        # temp_lable.append('[SEP]')
        # temp_token.append('[SEP]')
        
        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)
    
    id_list = []
    target_list = [] 
    attention_mask_list = []   
        
    # PADING        
    for text, label in zip(tokenized_texts, word_piece_labels):
        
        # Add [CLS] and [SEP], 
        # Truncate seq if it is too long
        text = ['[CLS]'] + text[:max_len-2] + ['[SEP]']
        label = ['[CLS]'] + label[:max_len-2] + ['[SEP]']
        
        # convert to ids
        ids = tokenizer.convert_tokens_to_ids(text)
        target_tag =[tag2idx.get(t) for t in label]
        
        # padding 
        # Label [PAD] with O (other)
        padding_len = max_len - len(ids)
        ids = ids + [0] * padding_len
        target_tag = target_tag + [tag2idx['O']] * padding_len
        
        # create masks
        attention_masks = [int(i>0) for i in ids]
        
        id_list.append(ids)
        target_list.append(target_tag)
        attention_mask_list.append(attention_masks)
    
    return id_list, target_list, attention_mask_list

    
    
    

In [21]:
train_input_ids, train_input_tags, train_attention_masks = process_data(train_sentences, train_labels)
dev_input_ids, dev_input_tags, dev_attention_masks = process_data(dev_sentences, dev_labels)
test_input_ids, test_input_tags, test_attention_masks = process_data(test_sentences, test_labels)

In [22]:
train_inputs = torch.tensor(train_input_ids)
dev_inputs = torch.tensor(dev_input_ids)
test_inputs = torch.tensor(test_input_ids)
train_tags = torch.tensor(train_input_tags)
dev_tags = torch.tensor(dev_input_tags)
test_tags = torch.tensor(test_input_tags)
train_masks = torch.tensor(train_attention_masks)
dev_masks = torch.tensor(dev_attention_masks)
test_masks = torch.tensor(test_attention_masks)

**Put data into data loader**

In [23]:
# Set batch num
batch_num = 32

In [24]:
# Only set token embedding, attention embedding, no segment embedding
train_dataset = TensorDataset(train_inputs, train_masks, train_tags)
train_sampler = RandomSampler(train_dataset)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_num, drop_last=True)

dev_dataset = TensorDataset(dev_inputs, dev_masks, dev_tags)
dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_num)

test_data = TensorDataset(test_inputs, test_masks, test_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num)

## Train model

**Load BERT model**

In [25]:
# In this folder, contain model confg(json) and model weight(bin) files
# pytorch_model.bin, download from: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-pytorch_model.bin
# config.json, downlaod from: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json
model_file_address = model_name

In [26]:
# Will load config and weight with from_pretrained()
#model = BertForTokenClassification.from_pretrained(model_file_address, num_labels=len(tag2idx))

model = BertForTokenClassification.from_pretrained(model_file_address, num_labels=len(tag2idx))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Set model to GPU,if you are using GPU machine
# model.cuda();
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [28]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [29]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [30]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int(math.ceil(len(train_inputs) / batch_num) / 1) * epochs

### Set fine tuning method

**Manual optimizer**

In [31]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [32]:
# Optimizer setup
param_optimizer = list(model.named_parameters())
if FULL_FINETUNING:
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(optimizer_grouped_parameters, lr=5e-5)

### Fine-tuning model

In [33]:
# TRAIN loop
model.train()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [34]:
print("***** Running training *****")
print("  Num examples = %d"%(len(train_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for batch in train_dataloader:
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss  # Loss is explicitly available in newer versions
        scores = outputs.logits  # Access logits for predictions

        if n_gpu > 1:
            # When using multiple GPUs, average the loss
            loss = loss.mean()

        # backward pass
        loss.backward()

        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

    # print train loss per epoch
    print(f"Train loss: {tr_loss / nb_tr_steps}")
        

***** Running training *****
  Num examples = 81
  Batch size = 32
  Num steps = 15


Epoch:  20%|██        | 1/5 [00:43<02:55, 43.75s/it]

Train loss: 1.9677584767341614


Epoch:  40%|████      | 2/5 [01:23<02:04, 41.43s/it]

Train loss: 1.767440676689148


...

Train loss: 1.5597438216209412


Epoch:  80%|████████  | 4/5 [02:28<00:35, 35.45s/it]

Train loss: 1.367362380027771


Epoch: 100%|██████████| 5/5 [03:01<00:00, 36.31s/it]

Train loss: 1.1403502225875854


## Save model 

In [35]:
# Extract the base model name (remove any slashes or paths)
model_base_name = model_name.split("/")[-1]

In [36]:
# Define the output address based on the model name
bert_out_address = f"../trained_models/NER/{model_base_name}-test"

In [37]:
import shutil

# Check if the directory exists
if os.path.exists(bert_out_address):
    # Delete the directory and its contents
    shutil.rmtree(bert_out_address)

# Now create the directory
os.makedirs(bert_out_address)

In [38]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [39]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

In [40]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('../trained_models/NER/biobert-base-cased-v1.1-test\\vocab.txt',)

## Load model

In [41]:
model = BertForTokenClassification.from_pretrained(bert_out_address, num_labels=len(tag2idx))

In [42]:
# Set model to GPU
# model.cuda();
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [43]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

## Eval model

In [44]:
# Evalue loop
model.eval();

In [45]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print(f"  Num examples = {len(test_inputs)}")
print(f"  Batch size = {batch_num}")
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits, dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []

        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] not in ["X", "[CLS]", "[SEP]"]: # Exclude the X label
                    # print(tag2name[logits[i][j]])
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break

        y_true.append(temp_1)
        y_pred.append(temp_2)


print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))



***** Running evaluation *****
  Num examples = 60
  Batch size = 32
f1 socre: 0.000000
Accuracy score: 0.732000


In [46]:
classification_report(y_true, y_pred, digits=4)

c:\Users\nipua\AppData\Local\anaconda3\envs\ner_ast\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nipua\AppData\Local\anaconda3\envs\ner_ast\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n     problem     0.0000    0.0000    0.0000        30\n        test     0.0000    0.0000    0.0000        11\n   treatment     0.0000    0.0000    0.0000        25\n\n   micro avg     0.0000    0.0000    0.0000        66\n   macro avg     0.0000    0.0000    0.0000        66\nweighted avg     0.0000    0.0000    0.0000        66\n'

In [47]:
# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred, digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Eval results *****

              precision    recall  f1-score   support

     problem     0.0000    0.0000    0.0000        30
        test     0.0000    0.0000    0.0000        11
   treatment     0.0000    0.0000    0.0000        25

   micro avg     0.0000    0.0000    0.0000        66
   macro avg     0.0000    0.0000    0.0000        66
weighted avg     0.0000    0.0000    0.0000        66

f1 socre: 0.000000
Accuracy score: 0.732000
